Guignard Quentin, Stone Tomas, Hanus Maxime

# Question 1 : Explication de type (1-1) pour la Comparaison x > y

## Formulation Mathématique

### 1. Données et Paramètres
- **x, y** : vecteurs de notes des deux candidats pour chaque cours i
- **w** : vecteur des poids de chaque cours
- **contributions(i)** = w[i] × (x[i] - y[i]) : contribution du cours i à la somme pondérée

### 2. Ensembles
- **pros(x,y)** : ensemble des cours i où x[i] > y[i] (contributions > 0)
- **cons(x,y)** : ensemble des cours i où x[i] < y[i] (contributions < 0)
- **neutral(x,y)** : ensemble des cours i où x[i] = y[i] (contributions = 0)

### 3. Formulation du PL

**Variables de décision:**
- Pour chaque paire (P, C) où P \in pros(x,y) et C \in cons(x,y) :
  - $x_{P,C} \in {0, 1}$ : égal à 1 si la paire (P, C) est sélectionnée comme trade-off

**Contraintes:**
1. Chaque cours $C \in \text{cons}(x,y)$ doit être appairé exactement une fois :
   $$\sum_{P \in \text{pros}(x,y)} x_{P,C} = 1 \quad \forall C \in \text{cons}(x,y)$$

2. Chaque cours $P \in \text{pros}(x,y)$ peut être appairé au plus une fois :
   $$\sum_{C \in \text{cons}(x,y)} x_{P,C} \leq 1 \quad \forall P \in \text{pros}(x,y)$$

3. Chaque trade-off sélectionné doit être valide (contribution(P) + contribution(C) > 0) :
   $$(\text{contribution}(P) + \text{contribution}(C)) \cdot x_{P,C} > 0$$
   
   Ou de façon linéaire (si toutes les paires positives) :
   $$\text{contribution}(P) + \text{contribution}(C) > 0 \quad \text{pour chaque paire sélectionnée}$$

**Objectif:**
- Minimiser le nombre de trade-offs : $\min \sum_{P,C} x_{P,C}$
- Ou maximiser pour chercher une explication quelconque : $\max \sum_{P,C} x_{P,C}$

**Certificat de non-existence:**
Si le PL est infaisable (aucune solution), il n'existe pas d'explication (1-1).

In [18]:
import numpy as np
from gurobipy import *

# Données
courses = ['Anatomie', 'Biologie', 'Chirurgie', 'Diagnostic', 'Epidemiologie', 'Forensic', 'Genetique']
x_notes = np.array([85, 81, 71, 69, 75, 81, 88])
y_notes = np.array([81, 81, 75, 63, 67, 88, 95])
z_notes = np.array([74, 89, 74, 81, 68, 84, 79])
t_notes = np.array([74, 71, 84, 91, 77, 76, 73])
u_notes = np.array([72, 75, 66, 85, 88, 66, 93])
v_notes = np.array([71, 73, 63, 92, 76, 79, 93])
w_notes = np.array([79, 69, 78, 76, 67, 84, 79])
w_prime_notes = np.array([57, 76, 81, 76, 82, 86, 77])
weights = np.array([8, 7, 7, 6, 6, 5, 6])

# Contributions pondérées
contributions = weights * (x_notes - y_notes)
pros = [i for i in range(len(courses)) if contributions[i] > 0]
cons = [i for i in range(len(courses)) if contributions[i] < 0]
neutral = [i for i in range(len(courses)) if contributions[i] == 0]

print('pros:  ', [courses[i] for i in pros])
print('cons:  ', [courses[i] for i in cons])
print('neutral:', [courses[i] for i in neutral])

# Trade-offs (1-1) valides: contribution(P)+contribution(C) > 0
valid_tradeoffs = []
for p in pros:
    for c in cons:
        s = contributions[p] + contributions[c]
        if s > 0:
            valid_tradeoffs.append((p, c, s))

print('trade-offs valides:', [(courses[p], courses[c], s) for p, c, s in valid_tradeoffs])

pros:   ['Anatomie', 'Diagnostic', 'Epidemiologie']
cons:   ['Chirurgie', 'Forensic', 'Genetique']
neutral: ['Biologie']
trade-offs valides: [('Anatomie', 'Chirurgie', 4), ('Diagnostic', 'Chirurgie', 8), ('Diagnostic', 'Forensic', 1), ('Epidemiologie', 'Chirurgie', 20), ('Epidemiologie', 'Forensic', 13), ('Epidemiologie', 'Genetique', 6)]


In [19]:
# Résolution PL pour une explication (1-1) minimale
m = Model('explication_1_1')
x_vars = {}
for p, c, _ in valid_tradeoffs:
    x_vars[(p,c)] = m.addVar(vtype=GRB.BINARY, name=f'x_{p}_{c}')
m.update()

# Chaque cons est couvert exactement une fois
for c in cons:
    pairs = [(p, cc) for (p, cc, _) in valid_tradeoffs if cc == c]
    if pairs:
        m.addConstr(quicksum(x_vars[(p, cc)] for (p, cc) in pairs) == 1)

# Chaque pro est utilisé au plus une fois
for p in pros:
    pairs = [(pp, c) for (pp, c, _) in valid_tradeoffs if pp == p]
    if pairs:
        m.addConstr(quicksum(x_vars[(pp, c)] for (pp, c) in pairs) <= 1)

# Objectif: minimiser le nombre de paires
m.setObjective(quicksum(x_vars.values()), GRB.MINIMIZE)
m.params.outputflag = 0
m.optimize()

if m.status == GRB.OPTIMAL:
    print('OPTIMAL, longueur =', int(m.objVal))
    chosen = []
    for (p, c), var in x_vars.items():
        if var.X > 0.5:
            chosen.append((p, c))
            pc = contributions[p]
            cc = contributions[c]
            print(f'({courses[p]}, {courses[c]}) -> {pc} + ({cc}) = {pc+cc}')
else:
    print('Pas de solution (certificat de non-existence).')

OPTIMAL, longueur = 3
(Anatomie, Chirurgie) -> 32 + (-28) = 4
(Diagnostic, Forensic) -> 36 + (-35) = 1
(Epidemiologie, Genetique) -> 48 + (-42) = 6


# Question 2 : Explication de type (1-m) pour la Comparaison x > y

## Formulation Mathématique

### 1. Définition d'un Trade-off (1-m)
Un trade-off (1-m) est une paire (P, {C₁, ..., Cₘ}) où :
- P $\in$ pros(x,y) : un cours où x fait mieux que y
- {C₁, ..., Cₘ} $\subset$ cons(x,y) : m cours où y fait mieux que x
- La somme des contributions est positive : contribution(P) + Σⱼ contribution(Cⱼ) > 0

### 2. Explication (1-m)
Une explication (1-m) de x > y est un ensemble de trade-offs (1-m) disjoints qui couvrent tous les cons(x,y).

### 3. Formulation du Programme Linéaire

**Variables de décision:**
- Pour chaque P $\in$ pros(x,y) et chaque C \in cons(x,y) :
  - $x_{P,C} \in \{0, 1\}$ : égal à 1 si le cours C est associé au cours P dans un trade-off

**Contraintes:**
1. Chaque cours $C \in \text{cons}(x,y)$ doit être appairé exactement une fois :
   $$\sum_{P \in \text{pros}(x,y)} x_{P,C} = 1 \quad \forall C \in \text{cons}(x,y)$$

2. Pour chaque cours $P \in \text{pros}(x,y)$, si P est utilisé, son trade-off doit être valide :
   $$\text{contribution}(P) + \sum_{C \in \text{cons}(x,y)} \text{contribution}(C) \cdot x_{P,C} > 0$$
   
   Version linéarisée avec $y_P \in \{0,1\}$ (indicateur si P est utilisé) :
   $$\text{contribution}(P) \cdot y_P + \sum_{C \in \text{cons}(x,y)} \text{contribution}(C) \cdot x_{P,C} \geq \epsilon \cdot y_P$$
   
   où $\epsilon > 0$ est une petite constante (e.g., 0.001).

3. Lien entre $x_{P,C}$ et $y_P$ :
   $$x_{P,C} \leq y_P \quad \forall P \in \text{pros}(x,y), C \in \text{cons}(x,y)$$
   $$y_P \leq \sum_{C \in \text{cons}(x,y)} x_{P,C} \quad \forall P \in \text{pros}(x,y)$$

**Objectif:**
- Minimiser le nombre de trade-offs : $\min \sum_{P \in \text{pros}(x,y)} y_P$

**Certificat de non-existence:**
Si le PL est infaisable, il n'existe pas d'explication (1-m).

In [20]:
# Implémentation du PL pour explication (1-m)
def find_explanation_1_m(x_notes, y_notes, weights, courses, candidate_x='X', candidate_y='Y'):
    """
    Trouve une explication (1-m) minimale pour x > y
    Retourne: (status, explanation) où status est 'OPTIMAL' ou 'INFEASIBLE'
    """
    # Contributions pondérées
    contributions = weights * (x_notes - y_notes)
    pros = [i for i in range(len(courses)) if contributions[i] > 0]
    cons = [i for i in range(len(courses)) if contributions[i] < 0]
    
    print(f'\n=== Analyse {candidate_x} > {candidate_y} ===')
    print(f'pros({candidate_x},{candidate_y}):  ', [courses[i] for i in pros])
    print(f'cons({candidate_x},{candidate_y}):  ', [courses[i] for i in cons])
    print(f'Contributions: {contributions}')
    
    if not cons:
        print(f'{candidate_x} domine {candidate_y} sur tous les critères.')
        return 'TRIVIAL', []
    
    if not pros:
        print(f'{candidate_y} domine {candidate_x} sur tous les critères.')
        return 'IMPOSSIBLE', []
    
    # Modèle PL
    m = Model('explication_1_m')
    m.params.outputflag = 0
    
    # Variables: x[p,c] = 1 si C est associé à P
    x_vars = {}
    for p in pros:
        for c in cons:
            x_vars[(p,c)] = m.addVar(vtype=GRB.BINARY, name=f'x_{p}_{c}')
    
    # Variables: y[p] = 1 si P est utilisé dans un trade-off
    y_vars = {}
    for p in pros:
        y_vars[p] = m.addVar(vtype=GRB.BINARY, name=f'y_{p}')
    
    m.update()
    
    epsilon = 0.001
    
    # Contrainte 1: Chaque cons est couvert exactement une fois
    for c in cons:
        m.addConstr(quicksum(x_vars[(p,c)] for p in pros) == 1, name=f'cover_cons_{c}')
    
    # Contrainte 2: Validité des trade-offs
    for p in pros:
        sum_contrib = contributions[p] * y_vars[p]
        for c in cons:
            sum_contrib += contributions[c] * x_vars[(p,c)]
        m.addConstr(sum_contrib >= epsilon * y_vars[p], name=f'valid_tradeoff_{p}')
    
    # Contrainte 3: Lien entre x et y
    for p in pros:
        for c in cons:
            m.addConstr(x_vars[(p,c)] <= y_vars[p], name=f'link1_{p}_{c}')
        m.addConstr(y_vars[p] <= quicksum(x_vars[(p,c)] for c in cons), name=f'link2_{p}')
    
    # Objectif: minimiser le nombre de trade-offs
    m.setObjective(quicksum(y_vars.values()), GRB.MINIMIZE)
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        print(f'\n Explication (1-m) trouvée, longueur = {int(m.objVal)}')
        explanation = []
        for p in pros:
            if y_vars[p].X > 0.5:
                cons_in_tradeoff = [c for c in cons if x_vars[(p,c)].X > 0.5]
                contrib_p = contributions[p]
                contrib_cons = [contributions[c] for c in cons_in_tradeoff]
                total = contrib_p + sum(contrib_cons)
                explanation.append((p, cons_in_tradeoff))
                print(f'  Trade-off: ({courses[p]}) vs {[courses[c] for c in cons_in_tradeoff]}')
                print(f'    Contributions: {contrib_p:.1f} + {contrib_cons} = {total:.1f}')
        return 'OPTIMAL', explanation
    else:
        print(f'\nPas d\'explication (1-m) (certificat de non-existence)')
        return 'INFEASIBLE', []

# Test avec Xavier vs Yvonne
result, expl = find_explanation_1_m(x_notes, y_notes, weights, courses, 'Xavier', 'Yvonne')


=== Analyse Xavier > Yvonne ===
pros(Xavier,Yvonne):   ['Anatomie', 'Diagnostic', 'Epidemiologie']
cons(Xavier,Yvonne):   ['Chirurgie', 'Forensic', 'Genetique']
Contributions: [ 32   0 -28  36  48 -35 -42]

 Explication (1-m) trouvée, longueur = 3
  Trade-off: (Anatomie) vs ['Chirurgie']
    Contributions: 32.0 + [-28] = 4.0
  Trade-off: (Diagnostic) vs ['Forensic']
    Contributions: 36.0 + [-35] = 1.0
  Trade-off: (Epidemiologie) vs ['Genetique']
    Contributions: 48.0 + [-42] = 6.0


## Démonstration : Absence d'explication (1-m) pour u > v

Nous allons maintenant prouver qu'il n'existe pas d'explication (1-m) pour la comparaison u > v.

In [21]:
# Test de l'explication (1-m) pour u > v
result_uv, expl_uv = find_explanation_1_m(u_notes, v_notes, weights, courses, 'u', 'v')


=== Analyse u > v ===
pros(u,v):   ['Anatomie', 'Biologie', 'Chirurgie', 'Epidemiologie']
cons(u,v):   ['Diagnostic', 'Forensic']
Contributions: [  8  14  21 -42  72 -65   0]

Pas d'explication (1-m) (certificat de non-existence)


## Explication de type (m-1) pour y > z

Un trade-off (m-1) est une paire ({P₁, ..., Pₘ}, C) où :
- {P₁, ..., Pₘ} $\subset$ pros(y,z) : m cours où y fait mieux que z
- C \in cons(y,z) : un cours où z fait mieux que y
- La somme des contributions est positive : Σⱼ contribution(Pⱼ) + contribution(C) > 0

### Formulation du Programme Linéaire pour (m-1)

**Variables:**
- $x_{P,C} \in \{0, 1\}$ : égal à 1 si P est associé au cons C
- $y_C \in \{0, 1\}$ : égal à 1 si C est utilisé dans un trade-off

**Contraintes:**
1. Chaque P \in pros(y,z) est associé à exactement un C :
   $$\sum_{C \in \text{cons}(y,z)} x_{P,C} = 1 \quad \forall P \in \text{pros}(y,z)$$

2. Validité des trade-offs (pour chaque C utilisé) :
   $$\sum_{P \in \text{pros}(y,z)} \text{contribution}(P) \cdot x_{P,C} + \text{contribution}(C) \cdot y_C \geq \epsilon \cdot y_C$$

3. Lien entre x et y :
   $$x_{P,C} \leq y_C \quad \forall P, C$$
   $$y_C \leq \sum_{P \in \text{pros}(y,z)} x_{P,C} \quad \forall C$$

**Objectif:** $\min \sum_C y_C$

In [22]:
# Implémentation du PL pour explication (m-1)
def find_explanation_m_1(x_notes, y_notes, weights, courses, candidate_x='X', candidate_y='Y'):
    """
    Trouve une explication (m-1) minimale pour x > y
    En (m-1), chaque cons doit être couvert par exactement un trade-off.
    Retourne: (status, explanation)
    """
    # Contributions pondérées
    contributions = weights * (x_notes - y_notes)
    pros = [i for i in range(len(courses)) if contributions[i] > 0]
    cons = [i for i in range(len(courses)) if contributions[i] < 0]
    
    print(f'\n=== Analyse {candidate_x} > {candidate_y} (type m-1) ===')
    print(f'pros({candidate_x},{candidate_y}):  ', [courses[i] for i in pros], 'contributions:', [contributions[i] for i in pros])
    print(f'cons({candidate_x},{candidate_y}):  ', [courses[i] for i in cons], 'contributions:', [contributions[i] for i in cons])
    
    if not cons:
        print(f'{candidate_x} domine {candidate_y} sur tous les critères.')
        return 'TRIVIAL', []
    
    if not pros:
        print(f'{candidate_y} domine {candidate_x} sur tous les critères.')
        return 'IMPOSSIBLE', []
    
    # Modèle PL
    m = Model('explication_m_1')
    m.params.outputflag = 0
    
    # Variables: x[p,c] = 1 si P est associé à C
    x_vars = {}
    for p in pros:
        for c in cons:
            x_vars[(p,c)] = m.addVar(vtype=GRB.BINARY, name=f'x_{p}_{c}')
    
    # Variables: y[c] = 1 si C est utilisé dans un trade-off (sera forcé à 1)
    y_vars = {}
    for c in cons:
        y_vars[c] = m.addVar(vtype=GRB.BINARY, name=f'y_{c}')
    
    m.update()
    
    epsilon = 0.0
    
    # Contrainte 1: Chaque cons DOIT être couvert (y_c = 1)
    for c in cons:
        m.addConstr(y_vars[c] == 1, name=f'cover_con_{c}')
    
    # Contrainte 2: Chaque pro est utilisé au plus une fois
    for p in pros:
        m.addConstr(quicksum(x_vars[(p,c)] for c in cons) <= 1, name=f'pro_used_once_{p}')
    
    # Contrainte 3: Validité des trade-offs
    # Pour chaque cons, la somme des contributions (cons + ses pros associés) doit être >= 0
    for c in cons:
        sum_contrib = contributions[c] * y_vars[c]
        sum_contrib += quicksum(contributions[p] * x_vars[(p,c)] for p in pros)
        m.addConstr(sum_contrib >= epsilon * y_vars[c], name=f'valid_tradeoff_{c}')
    
    # Contrainte 4: Lien entre x et y (y[c] contrôle quels x peuvent être non-zéro)
    for c in cons:
        for p in pros:
            m.addConstr(x_vars[(p,c)] <= y_vars[c], name=f'link1_{p}_{c}')
        m.addConstr(y_vars[c] <= quicksum(x_vars[(p,c)] for p in pros), name=f'link2_{c}')
    
    # Objectif: minimiser le nombre total de pros utilisés (plutôt que sum(y) qui est constant)
    m.setObjective(quicksum(x_vars.values()), GRB.MINIMIZE)
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        # Compter le nombre de trade-offs (= nombre de cons couverts = len(cons))
        num_tradeoffs = sum(1 for c in cons if y_vars[c].X > 0.5)
        print(f'\n Explication (m-1) trouvée, longueur = {num_tradeoffs}')
        explanation = []
        for c in cons:
            if y_vars[c].X > 0.5:
                pros_in_tradeoff = [p for p in pros if x_vars[(p,c)].X > 0.5]
                contrib_c = contributions[c]
                contrib_pros = [contributions[p] for p in pros_in_tradeoff]
                total = sum(contrib_pros) + contrib_c
                explanation.append((pros_in_tradeoff, c))
                print(f'  Trade-off: {[courses[p] for p in pros_in_tradeoff]} vs ({courses[c]})')
                print(f'    Contributions: {contrib_pros} + {contrib_c:.1f} = {total:.1f}')
        return 'OPTIMAL', explanation
    else:
        print(f'\n  Pas d\'explication (m-1) (certificat de non-existence)')
        return 'INFEASIBLE', []

# Test de l'explication (m-1) pour y > z
result_yz, expl_yz = find_explanation_m_1(y_notes, z_notes, weights, courses, 'y', 'z')


=== Analyse y > z (type m-1) ===
pros(y,z):   ['Anatomie', 'Chirurgie', 'Forensic', 'Genetique'] contributions: [56, 7, 20, 96]
cons(y,z):   ['Biologie', 'Diagnostic', 'Epidemiologie'] contributions: [-56, -108, -6]

 Explication (m-1) trouvée, longueur = 3
  Trade-off: ['Anatomie'] vs (Biologie)
    Contributions: [56] + -56.0 = 0.0
  Trade-off: ['Forensic', 'Genetique'] vs (Diagnostic)
    Contributions: [20, 96] + -108.0 = 8.0
  Trade-off: ['Chirurgie'] vs (Epidemiologie)
    Contributions: [7] + -6.0 = 1.0


# Question 3 : Explications Combinées (m-1) et (1-m)

## Formulation Mathématique

### 1. Définitions

Une **explication combinée** de x > y est un ensemble E = {(P₁^(m₁), C₁), ..., (Pₖ^(mₖ), Cₖ), (P'₁, C'₁^(n₁)), ..., (P'ₗ, C'ₗ^(nₗ))} où :
- Les k premiers sont des trade-offs de type (m-1) : chaque (Pᵢ^(mᵢ), Cᵢ) a mᵢ pros et 1 cons
- Les ℓ derniers sont des trade-offs de type (1-m) : chaque (P'ⱼ, C'ⱼ^(nⱼ)) a 1 pro et nⱼ cons
- Tous les cons(x,y) sont couverts exactement une fois
- Chaque trade-off est valide (somme des contributions > 0)

### 2. Formulation ILP Combinée

**Variables:**
- $x_{p,c} \in \{0, 1\}$ : pro p est associé au cons c dans un trade-off
- $z_c \in \{0, 1\}$ : cons c est utilisé dans un trade-off (type m-1 si z_c=1)
- $w_p \in \{0, 1\}$ : pro p est utilisé dans un trade-off (type 1-m si w_p=1)
- $y_c \in \{0, 1\}$ : cons c est couvert (dans n'importe quel trade-off)

**Contraintes:**

1. **Couverture des cons** (exactement une fois) :
   $$\sum_{p \in pros} x_{p,c} = 1 \quad \forall c \in cons$$

2. **Chaque pro au plus une fois** :
   $$\sum_{c \in cons} x_{p,c} \leq 1 \quad \forall p \in pros$$

3. **Validité des trade-offs** :
   $$\sum_{p \in pros} \omega_p \cdot x_{p,c} + \omega_c \geq 0 \quad \forall c \in cons$$

4. **Activation des indicateurs** :
   - Si c est couvert : $y_c = 1$
   - Si c est en (m-1) : $z_c \leq \sum_p x_{p,c}$
   - Si c est en (1-m) : $w_p \leq \sum_c x_{p,c}$

**Objectif:**
- Minimiser la longueur totale : $\min \sum_c y_c$

### 3. Analyse de z > t

Pour z > t, nous allons montrer :
1. Qu'il n'existe pas d'explication (1-m) seule
2. Qu'il n'existe pas d'explication (m-1) seule
3. Qu'une explication combinée existe

In [24]:
# Vérification pour z > t : pas de (1-m) ni (m-1) seule
print('\n' + '='*80)
print('QUESTION 3 : Analyse de z > t')
print('='*80)

print('\n--- Test (1-m) pour z > t ---')
result_1m, expl_1m = find_explanation_1_m(z_notes, t_notes, weights, courses, 'z', 't')

print('\n--- Test (m-1) pour z > t ---')
result_m1, expl_m1 = find_explanation_m_1(z_notes, t_notes, weights, courses, 'z', 't')


QUESTION 3 : Analyse de z > t

--- Test (1-m) pour z > t ---

=== Analyse z > t ===
pros(z,t):   ['Biologie', 'Forensic', 'Genetique']
cons(z,t):   ['Chirurgie', 'Diagnostic', 'Epidemiologie']
Contributions: [  0 126 -70 -60 -54  40  36]

Pas d'explication (1-m) (certificat de non-existence)

--- Test (m-1) pour z > t ---

=== Analyse z > t (type m-1) ===
pros(z,t):   ['Biologie', 'Forensic', 'Genetique'] contributions: [126, 40, 36]
cons(z,t):   ['Chirurgie', 'Diagnostic', 'Epidemiologie'] contributions: [-70, -60, -54]

  Pas d'explication (m-1) (certificat de non-existence)


In [32]:
# Implémentation du PL pour explication mixte (m-1) + (1-m)
def find_explanation_mixed(x_notes, y_notes, weights, courses, candidate_x='X', candidate_y='Y'):
    """
    Trouve une explication mixte combinant (m-1) et (1-m) pour x > y
    - (m-1): plusieurs pros couvrent 1 cons
    - (1-m): 1 pro couvre plusieurs cons
    Retourne: (status, explanation)
    """
    x_notes = np.array(x_notes, dtype=float)
    y_notes = np.array(y_notes, dtype=float)
    weights = np.array(weights, dtype=float)

    omega = weights * (x_notes - y_notes)

    pros = [i for i in range(len(courses)) if omega[i] > 0]
    cons = [i for i in range(len(courses)) if omega[i] < 0]

    print(f'\n=== Analyse {candidate_x} > {candidate_y} (type mixte) ===')
    print(f'pros({candidate_x},{candidate_y}):  ', [courses[i] for i in pros], 'contributions:', [omega[i] for i in pros])
    print(f'cons({candidate_x},{candidate_y}):  ', [courses[i] for i in cons], 'contributions:', [omega[i] for i in cons])

    if not cons:
        print(f'{candidate_x} domine {candidate_y} sur tous les critères.')
        return 'TRIVIAL', []
    if not pros:
        print(f'{candidate_y} domine {candidate_x} sur tous les critères.')
        return 'IMPOSSIBLE', []

    M = float(np.sum(np.abs(omega)) + 1.0)
    epsilon = 0.0

    m = Model('explication_mixte')
    m.Params.OutputFlag = 0

    # Variables (m-1): x_pc[p,c] = 1 si pro p est associé au cons c dans un trade-off (m-1)
    x_pc = {(p,c): m.addVar(vtype=GRB.BINARY, name=f'x_{p}_{c}') for p in pros for c in cons}
    # y_c[c] = 1 si cons c est couvert par un trade-off (m-1)
    y_c = {c: m.addVar(vtype=GRB.BINARY, name=f'y_{c}') for c in cons}

    # Variables (1-m): z_pc[p,c] = 1 si cons c est associé au pro p dans un trade-off (1-m)
    z_pc = {(p,c): m.addVar(vtype=GRB.BINARY, name=f'z_{p}_{c}') for p in pros for c in cons}
    # u_p[p] = 1 si pro p est utilisé dans un trade-off (1-m)
    u_p = {p: m.addVar(vtype=GRB.BINARY, name=f'u_{p}') for p in pros}

    m.update()

    # Contrainte 1: Chaque cons couvert exactement une fois (soit par m-1 soit par 1-m)
    for c in cons:
        m.addConstr(y_c[c] + quicksum(z_pc[(p,c)] for p in pros) == 1, name=f'cover_{c}')

    # Contrainte 2: Cohérence et validité des trade-offs (m-1)
    for c in cons:
        for p in pros:
            m.addConstr(x_pc[(p,c)] <= y_c[c], name=f'link_m1_{p}_{c}')
        m.addConstr(quicksum(x_pc[(p,c)] for p in pros) >= y_c[c], name=f'nonempty_m1_{c}')
        m.addConstr(
            omega[c] + quicksum(omega[p] * x_pc[(p,c)] for p in pros) >= epsilon - M*(1 - y_c[c]),
            name=f'valid_m1_{c}'
        )

    # Contrainte 3: Cohérence et validité des trade-offs (1-m)
    for p in pros:
        for c in cons:
            m.addConstr(z_pc[(p,c)] <= u_p[p], name=f'link_1m_{p}_{c}')
        m.addConstr(quicksum(z_pc[(p,c)] for c in cons) >= u_p[p], name=f'nonempty_1m_{p}')
        m.addConstr(
            omega[p] + quicksum(omega[c] * z_pc[(p,c)] for c in cons) >= epsilon - M*(1 - u_p[p]),
            name=f'valid_1m_{p}'
        )

    # Contrainte 4: Pros disjoints (utilisé au plus une fois, pas dans les deux modes)
    for p in pros:
        m.addConstr(quicksum(x_pc[(p,c)] for c in cons) <= 1, name=f'pro_once_m1_{p}')
        m.addConstr(quicksum(x_pc[(p,c)] for c in cons) + u_p[p] <= 1, name=f'pro_disjoint_{p}')

    # Objectif: minimiser le nombre de trade-offs
    m.setObjective(quicksum(y_c[c] for c in cons) + quicksum(u_p[p] for p in pros), GRB.MINIMIZE)
    m.optimize()

    if m.Status != GRB.OPTIMAL:
        print(f'\nPas d\'explication mixte (certificat de non-existence)')
        return 'INFEASIBLE', []

    # Collecter les trade-offs
    explanation = []
    
    # Trade-offs (m-1)
    for c in cons:
        if y_c[c].X > 0.5:
            P = [p for p in pros if x_pc[(p,c)].X > 0.5]
            val = float(omega[c] + sum(omega[p] for p in P))
            explanation.append(('m-1', P, [c], val))

    # Trade-offs (1-m)
    for p in pros:
        if u_p[p].X > 0.5:
            Cset = [c for c in cons if z_pc[(p,c)].X > 0.5]
            val = float(omega[p] + sum(omega[c] for c in Cset))
            explanation.append(('1-m', [p], Cset, val))

    print(f'\n Explication mixte trouvée, longueur = {len(explanation)}')
    for typ, P, Cset, val in explanation:
        if typ == 'm-1':
            print(f'  Trade-off (m-1): {[courses[i] for i in P]} vs ({courses[Cset[0]]})')
            print(f'    Contributions: {[omega[i] for i in P]} + {omega[Cset[0]]:.1f} = {val:.1f}')
        else:
            print(f'  Trade-off (1-m): ({courses[P[0]]}) vs {[courses[i] for i in Cset]}')
            print(f'    Contributions: {omega[P[0]]:.1f} + {[omega[i] for i in Cset]} = {val:.1f}')

    return 'OPTIMAL', explanation

# Test de l'explication mixte pour z > t
print('\n' + '='*80)
print('TEST EXPLICATION MIXTE pour z > t')
print('='*80)

result_mixed, expl_mixed = find_explanation_mixed(z_notes, t_notes, weights, courses, 'z', 't')


TEST EXPLICATION MIXTE pour z > t

=== Analyse z > t (type mixte) ===
pros(z,t):   ['Biologie', 'Forensic', 'Genetique'] contributions: [126.0, 40.0, 36.0]
cons(z,t):   ['Chirurgie', 'Diagnostic', 'Epidemiologie'] contributions: [-70.0, -60.0, -54.0]

 Explication mixte trouvée, longueur = 2
  Trade-off (m-1): ['Forensic', 'Genetique'] vs (Chirurgie)
    Contributions: [40.0, 36.0] + -70.0 = 6.0
  Trade-off (1-m): (Biologie) vs ['Diagnostic', 'Epidemiologie']
    Contributions: 126.0 + [-60.0, -54.0] = 12.0
